# installs and imports

In [ ]:
from IPython.display import clear_output
!pip3 install pycaret --user
clear_output()

import numpy as np
import pandas as pd 

import random
import os 

from pycaret.regression import *

# global variables

In [ ]:
TRAIN_PATH = "../input/tabular-playground-series-jan-2022/train.csv"
TEST_PATH = "../input/tabular-playground-series-jan-2022/test.csv"
SAMPLE_SUBMISSION_PATH = "../input/tabular-playground-series-jan-2022/sample_submission.csv"
SUBMISSION_PATH = "submission.csv"

ID = "row_id"
TARGET = "num_sold"

FOLD = 2
BLEND_MODELS = ["rf","catboost","lightgbm"]
BLEND_METHOD = 'soft'
BLEND_FOLD = 2
CRITERIA = "mape"

TUNE_FOLD = 2

SEED = 2022

def seed_everything(seed: int = SEED):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    
seed_everything()

PYCARET_TARGET_NAME = "Label"

# load 

In [ ]:
train = pd.read_csv(TRAIN_PATH)
test = pd.read_csv(TEST_PATH)

# init pyCaret

In [ ]:
setup(
    data=train,
    target=TARGET,
    silent=True,
    ignore_features=[ID]
)

# Blend Models

In [ ]:
train_models = []
for model in BLEND_MODELS:
    trainedModel = create_model(model,fold = FOLD)
    train_models.append(trainedModel)
    

blendModel = blend_models(estimator_list = train_models,optimize=CRITERIA,fold = BLEND_FOLD)

In [ ]:
predict_model(blendModel)

# tune model

In [ ]:
tuneModel = tune_model(blendModel,fold=TUNE_FOLD)

In [ ]:
predict_model(tuneModel)

# best model

In [ ]:
final_model = finalize_model(tuneModel)

In [ ]:
predict_model(final_model)

# predict test data target 

In [ ]:
pred_test = predict_model(final_model, data=test)
pred_test[:5]

# make submission csv 

In [ ]:
sub = pd.read_csv(SAMPLE_SUBMISSION_PATH)
sub[TARGET] = pred_test[PYCARET_TARGET_NAME]
sub.to_csv(SUBMISSION_PATH,index=False)
sub.head()